In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -q -U roboflow
%pip install -r requirements.txt

import torch
import os
from IPython.display import Image, clear_output

print(f"Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 12694, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 12694 (delta 0), reused 1 (delta 0), pack-reused 12693
Receiving objects: 100% (12694/12694), 11.71 MiB | 11.66 MiB/s, done.
Resolving deltas: 100% (8853/8853), done.
/content/yolov5/yolov5/yolov5
Using torch 1.10.0+cu111 (Tesla V100-SXM2-16GB)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)
os.environ["DATASET_DIRECTORY"] = "/content/datasets"

Mounted at /content/drive


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="mRU23tAr1sKN2UOIokAG")
project = rf.workspace("umkc-490-spring-2022").project("weapon-detection-nicpz")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


KeyboardInterrupt: ignored

### IF STARTING A FRESH TRAINING RUN

In [ ]:
## Counterintuitively, ultralytics recommends higher batch size, at least 64.
## Also, the training automatically stops if loss is plateauing for 100 epochs.

!python train.py --img 736 --batch 64 --epochs 500 --data {dataset.location}/data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/datasets/Weapon-Detection-2/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=32, imgsz=1024, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-110-g0ca85ed torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, 

In [ ]:
#export your model's weights for future use
from google.colab import files
files.download('./runs/train/exp/weights/best.pt')
files.download('./runs/train/exp/weights/last.pt')

### FOR IMPORTING PRETRAINED MODELS

#### PRETRAINED DEMO MODEL

In [ ]:
!wget -P /content/yolov5/weights

#### LOCAL WEIGHT MODEL UPLOAD

In [ ]:
#Creating the directory
%mkdir /content/yolov5/weights

#modify the first part of path to wherever you have placed the best.pt in your drive
%cp /content/drive/My\ Drive/"INSERT PT FILE LOCATION HERE" /content/yolov5/weights

### DETECTION

#### LOCAL MODEL

In [ ]:
!python detect.py --weights /content/yolov5/weights/best.pt --img 736 --conf 0.1 --source "insert file name here"

#### USING PRE-SELECTED DATASET

In [ ]:
!python detect.py --weights /content/yolov5/weights/best.pt --img 736 --conf 0.1 --source {dataset.location}/test/images

detect: weights=['/content/yolov5/weights/best.pt'], source=/content/datasets/Weapon-Detection-2/test/images, data=data/coco128.yaml, imgsz=[720, 720], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-104-g741fac8 torch 1.10.0+cu111 CUDA:0 (Tesla V100-SXM2-16GB, 16160MiB)

Fusing layers... 
Model summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/499 /content/datasets/Weapon-Detection-2/test/images/DSC_0003_JPG.rf.83cc4dbefe07a1a1ec4c1039b7790d04.jpg: 512x736 2 knifes, Done. (0.020s)
image 2/499 /content/datasets/Weapon-Detection-2/test/images/DSC_0005_JPG.rf.057d5416de03906ade4729f1b61269af.jpg: 512x736 1 knife, Done. (0.011s)
image 3/499 /content/datasets/Weapon-D

### OUTPUT RESULTS

####IMAGES

In [ ]:
import glob

path_start = '/content/yolov5/'

print('Enter detection results path.')
file_name = input('It will be the last line from the previous cell after "saved to":')
path = path_start + file_name
full_path = path + '*.jpg'

for imageName in glob.glob(full_path):
  display(Image(filename=imageName))
  print("\n")

If multiple detection runs have been executed, you may have to change the file path to whatever is displayed in the previous run.